In [5]:
import numpy as np
import time
import os
import h5py
import inspect
from tqdm import tqdm
import sys
import math

import presto
from presto import lockin, utils
from presto.hardware import AdcFSample, AdcMode, DacFSample, DacMode


ModuleNotFoundError: No module named 'presto'

In [2]:
############################################################################
# Saving methods

# Save script function
def save_script(folder, file, sample, myrun, myrun_attrs):
    # Create folders if they do not exist
    if not os.path.isdir(folder):
        os.makedirs(folder)

    # String handles
    run_str = "{}/{}".format(sample, myrun)
    source_str = "{}/{}/Source code".format(sample, myrun)

    # Read lines of the script
    filename = inspect.getframeinfo(inspect.currentframe()).filename
    with open(filename, "r") as codefile:
        code_lines = codefile.readlines()

    # Write lines of script and run attributes
    with h5py.File(os.path.join(folder, file), "a") as savefile:

        dt = h5py.special_dtype(vlen=str)
        code_set = savefile.create_dataset(source_str.format(myrun), (len(code_lines),), dtype=dt)
        for i in range(len(code_lines)):
            code_set[i] = code_lines[i]

        # Save the attributes of the run
        for key in myrun_attrs:
            savefile[run_str].attrs[key] = myrun_attrs[key]

    # Debug
    print("Saved script and run attributes.")


# Save data function
def save_data(folder, file, sample, myrun, freq, freq_pump, pump_pwr, pump_phase, df, usb_arr):
    if not os.path.isdir(folder):
        os.makedirs(folder)

    # Open the save file (.hdf5) in append mode
    with h5py.File(os.path.join(folder, file), "a") as savefile:
        # String as handles
        freq_data_str = "{}/{}/freq comb".format(sample, myrun)
        freq_pump_data_str = "{}/{}/freq pumps".format(sample, myrun)
        pump_pwr_data_str = "{}/{}/pump pwr sweep".format(sample, myrun)
        pump_phase_data_str = "{}/{}/pump phase sweep".format(sample, myrun)
        df_data_str = "{}/{}/df".format(sample, myrun)
        usb_data_str = "{}/{}/USB".format(sample, myrun)

        # Write data to datasets
        savefile.create_dataset(freq_data_str, (np.shape(freq)),
                                dtype=float, data=freq)
        savefile.create_dataset(freq_pump_data_str, (np.shape(freq_pump)),
                                dtype=float, data=freq_pump)
        savefile.create_dataset(pump_pwr_data_str, (np.shape(pump_pwr)),
                                dtype=float, data=pump_pwr)
        savefile.create_dataset(pump_phase_data_str, (np.shape(pump_phase)),
                                dtype=float, data=pump_phase)
        savefile.create_dataset(df_data_str, (np.shape(df)),
                                dtype=float, data=df)
        savefile.create_dataset(usb_data_str, (np.shape(usb_arr)),
                                dtype=complex, data=usb_arr)

        # Write dataset attributes
        savefile[freq_data_str].attrs["Unit"] = "Hz"
        savefile[freq_pump_data_str].attrs["Unit"] = "Hz"
        savefile[pump_pwr_data_str].attrs["Unit"] = "fsu"
        savefile[pump_phase_data_str].attrs["Unit"] = "rad"
        savefile[df_data_str].attrs["Unit"] = "Hz"
        savefile[usb_data_str].attrs["Unit"] = "fsu complex"




In [3]:
Ym_str = time.strftime("%Y-%m")
save_folder = r'I:/JPA-Data/{}'.format(Ym_str)
save_file   = r"{}.hdf5".format(Ym_str)  
myrun       = time.strftime("%Y-%m-%d_%H_%M_%S")

sample      = 'JPA'
meas_type   = 'Scattering'
atten       = 80
temperature = 0.0107

# Type of measurement
reverse = False
fwd_and_rev = False

ADDRESS = '130.237.35.90'
PORT    = 42873
Box     = 'Presto DELTA'

input_port   = 1
output_port  = 1
flux_port    = 5
bias_port    = 4

# Pseudorandom noise (only when working with small amplitudes)
dither = False

# DAC current
DAC_CURRENT  = 32_000  # µA

# DC bias value
bias_val     = 1.5275   # FS     

# MEASUREMENT PARAMETERS
# NCO frequencies
fNCO = 4.1e9
fNCO_pump = 8.3e9
# Bandwidth in Hz
_df_arr = np.array([100e3]) # np.array([1e6, 750e3, 500e3, 250e3, 100e3, 75e3, 50e3, 25e3, 10e3, 7.5e3, 5e3, 2.5e3, 1e3]) #np.array([250e3, 100e3, 50e3, 10e3, 1e3]) 

# Number of pixels
_Npix = 5000
Npix_arr = _Npix*np.ones_like(_df_arr)
Navg = 1
# Number of pixels we discard
Nskip = 0
# Number of frequencies of the frequency comb
nr_sig_freqs = 81

# SIGNAL PARAMETERS
# Signal output amplitude from Presto [FS] (Never exceed 0.05FS for OUTPORT [input of JPA]!!! - actually 0.1 still is fine)
amp_sig = 0.04

# RF PUMP PARAMETERS
# Pump frequency in Hz
_fp_center = 8.4e9
# Number of pumps
nr_pump_freqs = 4
# Pump amplitude sweep (FS units between 0 and 1)
pump_amp_min = 0.0
pump_amp_max = 0.1
nr_pump_amps = 11
pump_amp_arr = np.linspace(pump_amp_min, pump_amp_max, nr_pump_amps)
k_array      = np.array([-9, -1, 1, 9])
# Pump 1 phase sweep
nr_pump_phases = 1
pump_phase_arr = np.linspace(0, 2 * np.pi, nr_pump_phases)

# Propagation Phase Drift
input_phases = np.zeros(nr_sig_freqs)
dPhiDrift_df = 1.8869636024e-6  # rad/Hz - Measured Drift in phase per Hz

# Instantiate lockin device
with lockin.Lockin(address=ADDRESS,
                   port=PORT,
                   adc_mode=AdcMode.Mixed,
                   adc_fsample=AdcFSample.G2,
                   dac_mode=[DacMode.Mixed02, DacMode.Mixed04, DacMode.Mixed02, DacMode.Mixed02],
                   dac_fsample=[DacFSample.G6, DacFSample.G10, DacFSample.G6, DacFSample.G6],
                   ) as lck:
    # Start timer
    t_start = time.strftime("%Y-%m-%d_%H_%M_%S")

    # Print Presto version
    print("Presto version: " + presto.__version__)

    # DAC current
    lck.hardware.set_dac_current(output_port, DAC_CURRENT)

    # Data
    usb_arr = np.zeros((len(_df_arr), nr_pump_amps, nr_pump_phases, nr_sig_freqs, nr_sig_freqs), dtype=complex)

    # Bandwidth array and frequency combs arrays (to save)
    df_arr = np.zeros_like(_df_arr)
    fs_comb_arr = np.zeros((len(df_arr), nr_sig_freqs))
    fp_comb_arr = np.zeros((len(df_arr), nr_pump_freqs))
    fp_center_arr = np.zeros_like(_df_arr)

    # Set DC bias value
    lck.hardware.set_dc_bias(bias_val, bias_port)
    lck.hardware.sleep(1.0, False)

    # Configure mixer for the JPA pump
    lck.hardware.configure_mixer(freq=fNCO_pump,
                                 out_ports=flux_port,
                                 sync=False,
                                 )

    # Configure mixer just to be able to create output and input groups
    lck.hardware.configure_mixer(freq=fNCO,
                                 in_ports=input_port,
                                 out_ports=output_port,
                                 sync=True,
                                 )

    # Create output group for the JPA pump frequency
    og_pump = lck.add_output_group(ports=flux_port, nr_freq=nr_pump_freqs)

    # Create output group for the signal
    og = lck.add_output_group(ports=output_port, nr_freq=1)
    og.set_amplitudes(amp_sig)
    og.set_phases(phases=0.0,
                  phases_q=-np.pi / 2)

    # Create input group
    ig = lck.add_input_group(port=input_port, nr_freq=nr_sig_freqs)

    # Add pseudorandom noise if needed
    lck.set_dither(dither, output_port)

    lck.apply_settings()
    lck.hardware.sleep(1e-4, False)

    # Display nice progress bar
    with tqdm(total=(len(_df_arr) * nr_pump_amps * nr_pump_phases * nr_sig_freqs), ncols=80) as pbar:

        # Bandwith loop
        for df_idx, df_val in enumerate(_df_arr):
            # Set measurement comb
            _fs_center = _fp_center / 2. - fNCO
            # Tune center frequency
            fs_center, df = lck.tune(_fs_center, df_val)
            # Listening comb. We listen at the signal, idler and half pump frequencies (and where we expect only noise)
            fs_comb = fs_center + df * np.linspace(int(-nr_sig_freqs / 2), int(nr_sig_freqs / 2), nr_sig_freqs)

            # Set pump frequency
            fp_center = 2 * (fs_center + fNCO) - fNCO_pump
            fp_comb = fp_center + df * k_array

            # Set df
            lck.set_df(df)

            # Save tuned df and the frequency combs
            df_arr[df_idx] = df
            fs_comb_arr[df_idx] = fs_comb
            fp_comb_arr[df_idx] = fp_comb
            fp_center_arr[df_idx] = fp_center

            # Set pump frequencies
            og_pump.set_frequencies(fp_comb)

            # Set signal frequencies
            ig.set_frequencies(fs_comb)

            # Set input phases
            input_phases[0]=0.0
            for i in range(1, nr_sig_freqs):
                input_phases[i] = input_phases[i-1] - dPhiDrift_df * df  # the minus work
            input_phases_q =  input_phases - np.pi /2 
            ig.set_phases(phases=input_phases.copy()) 

            # Pump amplitude loop
            for amp_idx, amp_val in enumerate(pump_amp_arr):
                # Pump amplitude
                og_pump.set_amplitudes(np.full(nr_pump_freqs, amp_val))

                # Pump 1 phase loop
                for phase_idx, phase_val in enumerate(pump_phase_arr):
                    phases_arr             = np.array([np.pi, 0, 0, 0])   #only one phase configuration
                    phases_q_arr           = phases_arr - np.pi/2
                    # phases_arr = np.zeros(nr_pump_freqs)
                    # phases_arr[0] = phase_val
                    # phases_q_arr = np.full(nr_pump_freqs, -np.pi / 2)
                    # phases_q_arr[0] = phase_val - np.pi / 2
                    og_pump.set_phases(phases=phases_arr,
                                       phases_q=phases_q_arr)

                    # Signal frequency loop
                    for sig_idx, sig_val in enumerate(fs_comb):
                        # Probe frequency
                        og.set_frequencies(sig_val)

                        lck.apply_settings()

                        # Get lock-in packets (pixels) from the local buffer
                        Npix = math.floor(Npix_arr[df_idx])
                        data = lck.get_pixels(Npix + Nskip, summed=False, nsum=Navg)
                        freqs, pixels_i, pixels_q = data[input_port]

                        # Convert a measured IQ pair into a low/high sideband pair
                        LSB, HSB = utils.untwist_downconversion(pixels_i, pixels_q)

                        usb_arr[df_idx, amp_idx, phase_idx, sig_idx] = np.mean(HSB[-Npix:], axis=0)

                        # Update progress bar
                        pbar.update(1)

    # Mute outputs at the end of the sweep
    og_pump.set_amplitudes(np.zeros(nr_pump_freqs))
    og.set_amplitudes(0.0)
    lck.apply_settings()
    lck.hardware.set_dc_bias(0.0, bias_port)

# Stop timer
t_end = time.strftime("%Y-%m-%d_%H_%M_%S")

# Create dictionary with attributes
myrun_attrs = {"Meas": meas_type,
               "Instr": Box,
               "T": temperature,
               "Sample": sample,
               "att": atten,
               "4K-amp_out": 42,
               "RT-amp_out": 41,
               "RT-amp_in": 0,
               "nr_sig_freqs": nr_sig_freqs,
               "nr_pump_freqs": nr_pump_freqs,
               "fp_center": fp_center_arr + fNCO_pump,
               "amp_sig": amp_sig,
               "DC bias": bias_val,
               "Npixels": Npix_arr,
               "Naverages": Navg,
               "Nskip": Nskip,
               "Dither": dither,
               "t_start": t_start,
               "t_meas": t_end,
               #"Script name": os.path.basename(__file__),
               }

# # Save script and attributes
# save_script(save_folder, save_file, meas_type, myrun, myrun_attrs)

# Save data
save_data(save_folder, save_file, meas_type, myrun, fs_comb_arr + fNCO, fp_comb_arr + fNCO_pump, pump_amp_arr,
          pump_phase_arr, df_arr, usb_arr)

Presto version: 2.16.0


100%|█████████████████████████████████████████| 891/891 [04:52<00:00,  3.05it/s]


In [4]:
# myrun       = time.strftime("%Y-%m-%d_%H_%M_%S")
# Save data
# save_data(save_folder, save_file, meas_type, myrun, fs_comb_arr + fNCO, fp_comb_arr + fNCO_pump, pump_amp_arr,
#           pump_phase_arr, df_arr, usb_arr)


## TESTING
import numpy as np
print(4.2e9 - 3.29e6*41)
print(8.4e9 - 3.29e6*9)

# Propagation Phase Drift
nr_sig_freqs = 81
input_phases = np.zeros(nr_sig_freqs)
dPhiDrift_df = 1.8869636024e-6  # rad/Hz - Measured Drift in phase per Hz
df=1e6

# Set input phases
input_phases[0]=0.0
for i in range(1, nr_sig_freqs):
    input_phases[i] = input_phases[i-1] + dPhiDrift_df * df
input_phases_q =  input_phases - np.pi /2 
print('dePhi/df: ',dPhiDrift_df)
print(input_phases)
print('df=1MHz  -->  dePhi_i: [',[input_phases[1]-input_phases[0],input_phases[2]-input_phases[1],input_phases[3]-input_phases[2],input_phases[4]-input_phases[3]],'...',[input_phases[-1]-input_phases[-2]],']')

4065110000.0
8370390000.0
dePhi/df:  1.8869636024e-06
[  0.           1.8869636    3.7739272    5.66089081   7.54785441
   9.43481801  11.32178161  13.20874522  15.09570882  16.98267242
  18.86963602  20.75659963  22.64356323  24.53052683  26.41749043
  28.30445404  30.19141764  32.07838124  33.96534484  35.85230845
  37.73927205  39.62623565  41.51319925  43.40016286  45.28712646
  47.17409006  49.06105366  50.94801726  52.83498087  54.72194447
  56.60890807  58.49587167  60.38283528  62.26979888  64.15676248
  66.04372608  67.93068969  69.81765329  71.70461689  73.59158049
  75.4785441   77.3655077   79.2524713   81.1394349   83.02639851
  84.91336211  86.80032571  88.68728931  90.57425292  92.46121652
  94.34818012  96.23514372  98.12210732 100.00907093 101.89603453
 103.78299813 105.66996173 107.55692534 109.44388894 111.33085254
 113.21781614 115.10477975 116.99174335 118.87870695 120.76567055
 122.65263416 124.53959776 126.42656136 128.31352496 130.20048857
 132.08745217 133.9744